In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/hww_simulated.csv")

In [4]:
df.drop(columns=["Particle.PT", "Particle.Eta", "Particle.Phi"], inplace=True)

In [5]:
df["Particle.PID"] = df["Particle.PID"].str[18:20]

In [6]:
df["Particle.PID"] = df["Particle.PID"].astype(int)

In [7]:
for col in df.columns[1:]:
    df[col] = df[col].str.replace("[", "")
    df[col] = df[col].str.replace("]", "")

In [8]:
print(df.head())

   Particle.PID Electron.PT Electron.Eta Electron.Phi    Muon.PT     Muon.Eta  \
0            13    44.94672   0.21702203    1.4967911  19.728607  -0.76609546   
1            11   32.669098   0.28524885     -0.93204  19.569141     0.722224   
2            11                                                                 
3            13                                                                 
4            13                                        25.512314   -1.6375457   

     Muon.Phi MissingET.MET MissingET.Eta MissingET.Phi  
0  -0.8761875     33.763885    0.21087119     -2.068956  
1   1.1187549     29.574873   -0.76974607      2.856485  
2                        0.            0.            0.  
3                 52.082302     2.2248452    -1.8333149  
4   1.6149265     24.645163     1.7733337    -1.1305432  


In [9]:
df.to_csv("data/hww_simulated_clean.csv", index=False)

In [10]:
df = pd.read_csv("data/hww_simulated_clean.csv")

In [11]:
from src.utils.change_of_coordinates import exp_to_four_vec

df[['Electron.E', 'Electron.px', 'Electron.py', 'Electron.pz']] = df.apply(
    lambda row: pd.Series(exp_to_four_vec(row['Electron.PT'], row['Electron.Eta'], row['Electron.Phi'], 0.000511)), axis=1)

df[['Muon.E', 'Muon.px', 'Muon.py', 'Muon.pz']] = df.apply(
    lambda row: pd.Series(exp_to_four_vec(row['Muon.PT'], row['Muon.Eta'], row['Muon.Phi'], 0.10566)), axis=1)

print(df.head())

   Particle.PID  Electron.PT  Electron.Eta  Electron.Phi    Muon.PT  Muon.Eta  \
0            13    44.946720      0.217022      1.496791  19.728607 -0.766095   
1            11    32.669098      0.285249     -0.932040  19.569141  0.722224   
2            11          NaN           NaN           NaN        NaN       NaN   
3            13          NaN           NaN           NaN        NaN       NaN   
4            13          NaN           NaN           NaN  25.512314 -1.637546   

   Muon.Phi  MissingET.MET  MissingET.Eta  MissingET.Phi  Electron.E  \
0 -0.876188      33.763885       0.210871      -2.068956   46.009344   
1  1.118755      29.574873      -0.769746       2.856485   34.007226   
2       NaN       0.000000       0.000000       0.000000         NaN   
3       NaN      52.082302       2.224845      -1.833315         NaN   
4  1.614926      24.645163       1.773334      -1.130543         NaN   

   Electron.px  Electron.py  Electron.pz     Muon.E    Muon.px    Muon.py  \
0  

In [12]:
# if "Particle.PID" == 11, create a new columns lep1 with the values of Electron
# if "Particle.PID" == 13, create a new columns lep1 with the values of Muon
# if "Particle.PID" == 11, create a new columns lep2 with the values of Muon
# if "Particle.PID" == 13, create a new columns lep2 with the values of Electron

df["p_l_1_E"] = df.apply(lambda row: row["Electron.E"] if row["Particle.PID"] == 11 else row["Muon.E"], axis=1)
df["p_l_1_x"] = df.apply(lambda row: row["Electron.px"] if row["Particle.PID"] == 11 else row["Muon.px"], axis=1)
df["p_l_1_y"] = df.apply(lambda row: row["Electron.py"] if row["Particle.PID"] == 11 else row["Muon.py"], axis=1)
df["p_l_1_z"] = df.apply(lambda row: row["Electron.pz"] if row["Particle.PID"] == 11 else row["Muon.pz"], axis=1)

df["p_l_2_E"] = df.apply(lambda row: row["Muon.E"] if row["Particle.PID"] == 11 else row["Electron.E"], axis=1)
df["p_l_2_x"] = df.apply(lambda row: row["Muon.px"] if row["Particle.PID"] == 11 else row["Electron.px"], axis=1)
df["p_l_2_y"] = df.apply(lambda row: row["Muon.py"] if row["Particle.PID"] == 11 else row["Electron.py"], axis=1)
df["p_l_2_z"] = df.apply(lambda row: row["Muon.pz"] if row["Particle.PID"] == 11 else row["Electron.pz"], axis=1)

df.drop(columns=["Electron.E", "Electron.px", "Electron.py", "Electron.pz", "Muon.E", "Muon.px", "Muon.py", "Muon.pz", "Particle.PID", "Electron.PT", "Electron.Eta", "Electron.Phi", "Muon.PT", "Muon.Eta", "Muon.Phi"], inplace=True)

In [13]:
print(df.head())

   MissingET.MET  MissingET.Eta  MissingET.Phi    p_l_1_E    p_l_1_x  \
0      33.763885       0.210871      -2.068956  25.806953  12.627984   
1      29.574873      -0.769746       2.856485  34.007226  19.477232   
2       0.000000       0.000000       0.000000        NaN        NaN   
3      52.082302       2.224845      -1.833315        NaN        NaN   
4      24.645163       1.773334      -1.130543  68.079557  -1.125497   

     p_l_1_y    p_l_1_z    p_l_2_E   p_l_2_x    p_l_2_y    p_l_2_z  
0 -15.157571 -16.636398  46.009344  3.323257  44.823695   9.831179  
1 -26.227989   9.445712  24.898808  8.547856  17.603563  15.394421  
2        NaN        NaN        NaN       NaN        NaN        NaN  
3        NaN        NaN        NaN       NaN        NaN        NaN  
4  25.487476 -63.118434        NaN       NaN        NaN        NaN  


In [14]:
import numpy as np

df["mpx"] = df.apply(lambda row: row["MissingET.MET"] * np.cos(row["MissingET.Phi"]), axis=1)
df["mpy"] = df.apply(lambda row: row["MissingET.MET"] * np.sin(row["MissingET.Phi"]), axis=1)

In [15]:
df.drop(columns=["MissingET.MET", "MissingET.Phi", "MissingET.Eta"], inplace=True)

In [16]:
df_truth = pd.read_csv("data/df_final.csv")

df["p_v_1_E"] = df_truth["p_v_1_E"]
df["p_v_1_x"] = df_truth["p_v_1_x"]
df["p_v_1_y"] = df_truth["p_v_1_y"]
df["p_v_1_z"] = df_truth["p_v_1_z"]

df["p_v_2_E"] = df_truth["p_v_2_E"]
df["p_v_2_x"] = df_truth["p_v_2_x"]
df["p_v_2_y"] = df_truth["p_v_2_y"]
df["p_v_2_z"] = df_truth["p_v_2_z"]

In [17]:
# Cast NaN to 0
df.fillna(0, inplace=True)

df.to_csv("data/hww_simulated_omnifold.csv", index=False)

print(df.head())

     p_l_1_E    p_l_1_x    p_l_1_y    p_l_1_z    p_l_2_E   p_l_2_x    p_l_2_y  \
0  25.806953  12.627984 -15.157571 -16.636398  46.009344  3.323257  44.823695   
1  34.007226  19.477232 -26.227989   9.445712  24.898808  8.547856  17.603563   
2   0.000000   0.000000   0.000000   0.000000   0.000000  0.000000   0.000000   
3   0.000000   0.000000   0.000000   0.000000   0.000000  0.000000   0.000000   
4  68.079557  -1.125497  25.487476 -63.118434   0.000000  0.000000   0.000000   

     p_l_2_z        mpx        mpy     p_v_1_E    p_v_1_x    p_v_1_y  \
0   9.831179 -16.132711 -29.660336   24.155747  -9.751504   1.652740   
1  15.394421 -28.380976   8.318251  104.635343 -33.046353  -6.449344   
2   0.000000   0.000000   0.000000  454.164621  10.101091  36.836577   
3   0.000000 -13.516069 -50.297933   11.520140   0.308590  -9.489610   
4   0.000000  10.502991 -22.295094  518.828622   6.080513 -24.439560   

      p_v_1_z     p_v_2_E    p_v_2_x    p_v_2_y     p_v_2_z  
0  -22.038074   32